In [ ]:
!pip install chromadb

In [61]:
import chromadb

client = chromadb.Client()
collection = client.create_collection("fruit")

InternalError: Collection [fruit] already exists

In [ ]:
# Creating a collection by passing documents as input together with metadata
# Chomadb uses "Sentence transformers" to create it own embeddings
collection.add(
    ids=["1", "2", "3", "4", "5", "6", "7"],
    documents=["apple",
        "banana",
        "pineapple",
        "mango",
        "dragonfruit",
        "passionfruit",
        "raspberry"],
    metadatas=[
        { "color": "red", "weight": 180 },
        { "color": "yellow", "weight": 120 },
        { "color": "brown", "weight": 900 },
        { "color": "yellow", "weight": 200 },
        { "color": "pink", "weight": 600 },
        { "color": "purple", "weight": 18 },
        { "color": "red", "weight": 4 },        
    ]
)

In [ ]:
# Browsing the collection
# Print everything
all_data = collection.get()
print(all_data)

In [ ]:
from pprint import pprint
pprint(all_data)

In [ ]:
# NOTE - embeddings : None, Older version of Chroma use to use default Sentence transformers, but as of 2025 it is skipped
# You need to explicity specify

# Clearing the collection
client.delete_collection("fruit")

In [ ]:
# Using OpenAI embedding function
!pip install openai
!pip install python-dotenv

In [ ]:
import openai
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from dotenv import load_dotenv

In [ ]:
load_dotenv()
embedding_func = OpenAIEmbeddingFunction(
    model_name="text-embedding-3-small"  # or "text-embedding-3-large"
)

In [ ]:
collection = client.create_collection("fruit", embedding_function=embedding_func)

In [ ]:
collection.add(
    ids=["1", "2", "3", "4", "5", "6", "7"],
    documents=["apple",
        "banana",
        "pineapple",
        "mango",
        "dragonfruit",
        "passionfruit",
        "raspberry"],
    metadatas=[
        { "color": "red", "weight": 180 },
        { "color": "yellow", "weight": 120 },
        { "color": "brown", "weight": 900 },
        { "color": "yellow", "weight": 200 },
        { "color": "pink", "weight": 600 },
        { "color": "purple", "weight": 18 },
        { "color": "red", "weight": 4 },        
    ]
)

In [ ]:
all_data = collection.get()
pprint(all_data)

In [ ]:
# Gotcha moment!
# Embeddings are not returned by default to save bandwidth
# Do the following to show embeddings
all_data = collection.get(include=["embeddings", "documents", "metadatas"])
pprint(all_data)

In [62]:
# Now that data is stored with embeddings, lets query and so semantic search!
# Semantic Search
tropical_fruits = collection.query(query_texts=["Tropical"])
pprint(tropical_fruits)

{'data': None,
 'distances': [[0.6120957732200623,
                0.6204509735107422,
                0.6238695979118347,
                0.6701418161392212,
                0.706540584564209,
                0.7711185216903687,
                0.7747601866722107]],
 'documents': [['pineapple',
                'mango',
                'passionfruit',
                'dragonfruit',
                'banana',
                'apple',
                'raspberry']],
 'embeddings': None,
 'ids': [['3', '4', '6', '5', '2', '1', '7']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[{'color': 'brown', 'weight': 900},
                {'color': 'yellow', 'weight': 200},
                {'color': 'purple', 'weight': 18},
                {'color': 'pink', 'weight': 600},
                {'color': 'yellow', 'weight': 120},
                {'color': 'red', 'weight': 180},
                {'color': 'red', 'weight': 4}]],
 'uris': None}


In [ ]:
# Print required data
print(tropical_fruits["documents"][0])

In [63]:
# Metadata Filtering
tropical_yellow = collection.query(query_texts=["Tropical"], 
                                  where={"$and":[
                                      {"color": "yellow"},
                                      {"weight": {"$gt": 150}}
                                  ]
                                  })

In [ ]:
print(tropical_yellow["documents"][0])

In [ ]:
# Full Text Search
contains_fruit = collection.get(
    where_document={"$contains": "fruit"}
)
print(contains_fruit["documents"])

In [ ]:
# Types of Storage
# Ephemeral Client - What we did so far, NOT persisted
# Persistent Client - Saved to Disk
# Cloud Client - Saved to Cloud

In [ ]:
# We will recreate the Fruits database with persistent client
persistent_directory = "db"
persistent_client = chromadb.PersistentClient(path=persistent_directory)

In [ ]:
persistent_client.delete_collection("fruit")

In [ ]:
persistent_collection = persistent_client.create_collection("fruit", embedding_function=embedding_func)

In [ ]:
persistent_collection.add(
    ids=["1", "2", "3", "4", "5", "6", "7"],
    documents=["apple",
        "banana",
        "pineapple",
        "mango",
        "dragonfruit",
        "passionfruit",
        "raspberry"],
    metadatas=[
        { "color": "red", "weight": 180 },
        { "color": "yellow", "weight": 120 },
        { "color": "brown", "weight": 900 },
        { "color": "yellow", "weight": 200 },
        { "color": "pink", "weight": 600 },
        { "color": "purple", "weight": 18 },
        { "color": "red", "weight": 4 },        
    ]
)

In [ ]:
!ls -al

In [ ]:
!tree